#helpful sources in 5PL and 4PL discussions
1) https://www.bio-rad.com/webroot/web/pdf/lsr/literature/Bulletin_3022.pdf
2) https://www.graphpad.com/guides/prism/latest/curve-fitting/reg_asymmetric_dose_response_ec.htm
3) https://www.mathworks.com/matlabcentral/fileexchange/38043-five-parameters-logistic-regression-there-and-back-again
4) spiess et al. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2386824/
5) https://stackoverflow.com/questions/55725139/fit-sigmoid-function-s-shape-curve-to-data-using-python
Autoregressive modeling and diagnostics for qPCR amplification
6) https://people.duke.edu/~ccc14/pcfb/analysis.html


In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.misc import derivative
import plotly.graph_objects as go
from scipy.optimize import curve_fit

In [2]:
def logistic5(x, a, b, c, d, g): # spiess et al. 5P logistic curve
    # a is ground (bottom) asymptote
    # b is slope
    # c is inflection point
    # d is maximum (top) asymptote
    # g is "...the additional asymmetry parameter. Setting parameter g = 1 yields the four-parameter curves..."
    numerator = a-d
    denominator = (1+(x/c)**b)**g
    return (d+(numerator/denominator))

# provide initial ground_asymptote based on data
def ground_asymptote (fluor):
    # average first n cycles or so 
    # I think Anitoa does first 13 cycles
    n = 5
    return (sum(fluor[0:n]) / len(fluor[0:n]))

# provide initial ground_asymptote based on data
def max_asymptote (fluor):
    # average last n cycles or so 
    # I think Anitoa does first 13 cycles
    m = 5
    return (sum(fluor[-m:]) / len(fluor[-m:]))

# data for iconic, well-behaved rxn
iconic_y = [-23.72, -47.4, -27.93, -4.51, 0.65, 12.79, 29.96, 50.24, 79.85, 129.94, 203.74, 321.5, 488.96, 729.49, 1047.78, 1479.57, 1963.18, 2499.28, 3025.52, 3514.9, 3919.58, 4241.47, 4475.93, 4648.39, 4768.29, 4840.66, 4892.92, 4930.58, 4962.46, 4970.39, 4989.07, 4989.16, 4997.8, 4995.48, 5002.82, 5001.02, 5003.96, 5008.01, 5005.17, 5004.64, 5004.83]
cycles = np.arange(1,42)

# initial guesses: a, b, c, d, g
# p0 = [-7.498, 9.945, 18, 5013, 0.5451] #graphpad values with this dataset
# p0 = [4.04930663e+00, 6.34869459e+00, 2.12543050e+01, 5.00090309e+03, 2.31685219e+00] #scipy output using graphpad
p0 = (ground_asymptote(iconic_y), 10, 20, max_asymptote(iconic_y), 1) # g = 1 is no asymmetry

# parameters (optimal values), parameters covariance
popt, pcov = curve_fit(logistic5, cycles, iconic_y, p0) #least sq = dogbox
print (popt)
# calc opt fit
opt_y = logistic5(cycles, *popt)

# plot data vs opt fit
fig = go.Figure()
fig.add_trace(go.Scatter(x=cycles, y=iconic_y, mode='lines+markers', name='data', marker=dict(size=4, color='black'), line=dict(color='black', width=0.5)))
fig.add_trace(go.Scatter(x=cycles, y=opt_y, mode='lines+markers', name='fit', marker=dict(size=4, color='red'), line=dict(color='red', width=0.5)))

fig.update_layout(
    autosize=False,
    width=800,
    height=500, 
    title="Fitting qPCR Data vs Optimal Fit (Logistic 5PL)",
    xaxis_title="Cycle",
    yaxis_title="RFU",
    legend_title="Source",)
fig.show()


[4.04917778e+00 6.34869386e+00 2.12543067e+01 5.00090309e+03
 2.31685306e+00]


In [3]:
# calc 2nd derivative
second = []
for each in cycles:
    second.append(derivative(logistic5, x0=each, n=2, args=popt, dx=1e-6)) #numerical approx, not symbolic 2nd deriv
print (second)





[1.8189894035458565, 0.9094947017729282, 1.8189894035458565, -1.8189894035458565, 0.0, 8.185452315956354, 6.366462912410498, 12.732925824820995, 20.91837814077735, 33.651303965598345, 44.565240386873484, 55.47917680814862, 72.75957614183426, 82.76401786133647, 83.21876521222293, 70.9405867382884, 38.198777474462986, -2.2737367544323206, -41.8367562815547, -71.39533408917487, -83.6735125631094, -80.03553375601768, -69.12159733474255, -54.569682106375694, -38.198777474462986, -27.284841053187847, -18.189894035458565, -11.823431123048067, -8.185452315956354, -5.456968210637569, -2.7284841053187847, -2.7284841053187847, 0.0, -1.8189894035458565, -0.9094947017729282, -0.9094947017729282, 0.0, 0.9094947017729282, 0.0, 0.0, -0.9094947017729282]


In [4]:
# print (popt)
from sympy import *
x, a, b, c, d, g = symbols('x a b c d g')
# eqn = d+((a-d)/((1+(x/c)**b)**g))
firstD = diff(d+((a-d)/((1+(x/c)**b)**g)), x) #1st deriv
secondD = diff(d+((a-d)/((1+(x/c)**b)**g)), x, 2) #2nd deriv
secondD_with_subs = secondD.subs({a: 4.04930663e+00, b: 6.34869459e+00, c: 2.12543050e+01, d: 5.00090309e+03, g:2.31685219e+00 })

# secondD_with_subs.evalf(subs={x:20})
# secondD.evalf(subs={x: 20, a: 4.04930663e+00, b: 6.34869459e+00, c: 2.12543050e+01, d: 5.00090309e+03, g:2.31685219e+00 }) #-70.9297
secondDSubs_lambdified = lambdify(x, secondD_with_subs, "numpy")
secondDPoints = secondDSubs_lambdified(cycles)
# print (secondDPoints)

# plot 2nd deriv data
deriv = go.Figure()
deriv.add_trace(go.Scatter(x=cycles, y=second, mode='lines+markers', name='Taylor Approx', marker=dict(size=4, color='black'), line=dict(color='black', width=0.5)))
deriv.add_trace(go.Scatter(x=cycles, y=secondDPoints, mode='lines+markers', name='Symbolic Substitution', marker=dict(size=4, color='red'), line=dict(color='red', width=0.5)))

deriv.update_layout(
    autosize=False,
    width=800,
    height=500, 
    title="Second Derivative Plot of Logistic 5PL Using Two Methods",
    xaxis_title="Cycle",
    yaxis_title="RFU?",
    legend_title="Legend Title",)
deriv.show()


In [5]:
secondDSubs_scipyed = lambdify(x, secondD_with_subs, "scipy")
# print (secondDSubs_scipyed)
# localMin=optimize.minimize(secondDSubs_scipyed, x0=10)
# print (localMin)
# myMin = optimize.minimize_scalar(secondDSubs_scipyed)
# find global min
globMin = optimize.basinhopping(secondDSubs_scipyed, x0=20, stepsize=1)
print (globMin)
# grid = (0, 40)
# xmin_global = optimize.brute(secondDSubs_scipyed, (grid, ))
# print("Global minima found %s" % xmin_global)
# root = optimize.root(secondDSubs_scipyed, 18)  # our initial guess is 18; find max/min and find halfway pt
# print("First root found %s" % root.x)
# root2 = optimize.root(f, -2.5)
# print("Second root found %s" % root2.x)

# # Constrain optimization
# xmin_local = optimize.fminbound(f, 0, 10)
# print("Local minimum found %s" % xmin_local)

                        fun: -84.61956811609419
 lowest_optimization_result:       fun: -84.61956811609419
 hess_inv: array([[0.07303091]])
      jac: array([2.86102295e-06])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 5
     njev: 6
   status: 0
  success: True
        x: array([21.33218772])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 1158
                        nit: 100
                       njev: 579
                          x: array([21.33218772])
